In [4]:
import os
import sys
import pandas as pd
import numpy as np
import math

In [5]:
chunk_size = 100000
batch = 1
for chunk in pd.read_json('data/base/offers_corpus_english_v2.json', lines=True, nrows= 100000000000000, chunksize=chunk_size):
    chunk.to_json('data/base/product_corpus/chunk' + str(batch) + '.json')
    batch += 1

In [25]:
computer_df = pd.DataFrame()
for chunk in pd.read_json('data/base/offers_corpus_english_v2.json', lines=True, nrows= 100000000000000, chunksize=chunk_size):
    computer_df = computer_df.append(chunk[chunk['category'].values == 'Computers_and_Accessories'])

In [30]:
computer_df

,brand,category,cluster_id,description,id,identifiers,keyValuePairs,price,specTableContent,title
41,None,Computers_and_Accessories,1554982,None,41,"[{'/mpn': '[nxm81eh034]'}, {'/gtin13': '[47131...","{'categorie': 'laptops', 'merk': 'acer', 'prod...",None,categorie laptops merk acer productserie aspir...,acer aspire e1 522 65208g1tmnkk specificaties ...
55,None,Computers_and_Accessories,15189423,description,55,[{'/productID': '[k1009900]'}],None,None,None,kohler lavatory bonnet 1009900 bn ferguson
72,hp enterprise,Computers_and_Accessories,14583973,description hp third party rackmount option ki...,72,"[{'/sku': '[231122b21]'}, {'/mpn': '[231122b21...","{'category': 'hp option', 'sub category': 'rac...",None,specifications category hp option sub category...,"null , 231122 b21 hp 3rd party rail kit ml370 ..."
75,hp enterprise,Computers_and_Accessories,3859891,description hp proliant dl380 g6 rack mountabl...,75,[{'/sku': '[491505001]'}],None,None,None,"null , 491505 001 hp dl380 g6 e5504 2 00ghz 4g..."
91,None,Computers_and_Accessories,10106149,None,91,[{'/mpn': '[ds1010c101]'}],None,None,None,ds1010c 101
...,...,...,...,...,...,...,...,...,...,...
16451426,None,Computers_and_Accessories,4750388,None,17557407,[{'/gtin12': '[888462794961]'}],None,None,None,"""Apple MJYR2LL/A Smart Keyboard for 12.9-inch ..."
16451430,None,Computers_and_Accessories,518797,"""Unfold the full-size Smart Keyboard when you ...",17557412,[{'/mpn': '[mptl2lla]'}],None,None,None,"""Apple - Smart Keyboard for 10.5-inch iPad Pro..."
16451472,"""Apple""",Computers_and_Accessories,13868227,"""11.6-inch LED-backlit glossy widescreen TFT d...",17557460,[{'/mpn': '[mjvm2lla]'}],None,None,None,"""Macbook Air 11.6-inch (Glossy) 1.6GHZ Dual Co..."
16451484,None,Computers_and_Accessories,16080620,""" ...",17557479,[{'/productID': '[5403739000]'}],None,None,None,"""Acer Swift 1 SF113-31-P4A2 Pink ""@de"


In [51]:
computer_df = computer_df.drop_duplicates('title')

In [52]:
computer_df.to_csv('data/base/computer_wdc_whole_no_duplicates.csv', index=False)

In [13]:
def extract_key_features(computer_df):
    left = computer_df[['id_left', 'title_left',
                        'description_left', 'cluster_id_left']]
    right = computer_df[['id_right', 'title_right',
                         'description_right', 'cluster_id_right']]

    left = left.rename(columns={'id_left': 'id', 
                        'title_left': 'title',
                        'description_left': 'description',
                        'cluster_id_left': 'cluster_id'
                       })
    
    right = right.rename(columns={'id_right': 'id',
                          'title_right': 'title',
                          'description_right': 'description',
                          'cluster_id_right': 'cluster_id'
                         })
    
    ret = left.append(right)
    ret = ret.drop_duplicates(subset=['id'])
    
    return ret

In [14]:
extracted_df = extract_key_features(computer_df)

In [15]:
extracted_df

,id,title,description,cluster_id
0,2551242,"""Corsair Vengeance LPX Black 64GB (4x16GB) DD...","""DDR4, 2666MHz, CL16, 1.2v, XMP 2.0, Lifetime ...",79457
1,16757469,"""DH0072BALWL HP 72-GB 3G 15K 2.5 DP SAS"", ""Nu...","""Description:2 x 72GB 2.5-inch Serial Attached...",63553
2,232007,"""SanDisk SDSDJ-1024 BXP 1GB 9p SD Class 2 Sec...","""SDSDJ-1024 BXP 1GB 9p SD Class 2 Secure Digi...",212917
3,2066119,"""DISCO DURO INTERNO SOLIDO HDD SSD KINGSTON V...","""DISCO DURO INTERNO SOLIDO HDD SSD""@es",667772
4,6656540,"""Corsair Vengeance LED 32GB (2 x 16GB) DDR4 D...",None,505997
...,...,...,...,...
33509,14314785,"""case Logic LAPS116K 16"" Sleeve Negro maletin...","""40,6 cm Sleeve case Negro EVA Monótono ""@es",122211
34126,15885145,"""case Logic LAPS116K 16"" Sleeve Negro maletin...","""40,6 cm Sleeve case Negro EVA Monótono ""@es",122211
36399,9070035,"""case Logic LAPS116K 16"" Sleeve Negro maletin...","""40,6 cm Sleeve case Negro EVA Monótono ""@es",122211
43180,12849350,"""Verbatim 100 Pack CD-R White Printable Spind...","""The ultimate in reliability and recording per...",71929


In [48]:
all_clusters = set(new['cluster_id'].values)

In [36]:
num = 0
for cluster in all_clusters:
    if len(computer_df.loc[computer_df['cluster_id'].values == cluster]) > 20 and len(computer_df.loc[computer_df['cluster_id'].values == cluster]) < 80:
        num += 1

print(num)

3552


In [49]:
len(all_clusters)

295932